In [6]:
import pandas as pd
import MetaTrader5 as mt5
import matplotlib.pyplot as plt
import json

In [15]:
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()

tierminal_info = mt5.terminal_info()._asdict()
version= mt5.version()
print("MetaTrader5 version:", version)
df=pd.DataFrame(list(tierminal_info.items()),columns=['property','value'])
print(df)


MetaTrader5 version: (500, 4620, '11 Oct 2024')
                 property                                              value
0       community_account                                              False
1    community_connection                                              False
2               connected                                               True
3            dlls_allowed                                              False
4           trade_allowed                                               True
5       tradeapi_disabled                                              False
6           email_enabled                                              False
7             ftp_enabled                                              False
8   notifications_enabled                                              False
9                    mqid                                               True
10                  build                                               4620
11                maxbars   

In [16]:
account_info = mt5.account_info()._asdict()
df_account_info = pd.DataFrame(list(account_info.items()), columns=['property', 'value'])
print(df_account_info)

              property              value
0                login          312241304
1           trade_mode                  0
2             leverage               1000
3         limit_orders                300
4       margin_so_mode                  0
5        trade_allowed               True
6         trade_expert               True
7          margin_mode                  2
8      currency_digits                  2
9           fifo_close              False
10             balance           10022.72
11              credit                0.0
12              profit                0.0
13              equity           10022.72
14              margin                0.0
15         margin_free           10022.72
16        margin_level                0.0
17      margin_so_call               50.0
18        margin_so_so               20.0
19      margin_initial                0.0
20  margin_maintenance                0.0
21              assets                0.0
22         liabilities            

In [17]:
symbols=mt5.symbols_total()
print("Total symbols:",symbols)

Total symbols: 1505


In [19]:
symbols=mt5.symbols_get()

count=0
for s in symbols:
    count+=1
    print("{}. {}".format(count,s.name))
    if count==5: break
print()

1. EURUSD
2. GBPUSD
3. USDCHF
4. USDJPY
5. USDCAD



In [20]:
# get symbols containing RU in their names
ru_symbols=mt5.symbols_get("*RU*")
print('len(*RU*): ', len(ru_symbols))
for s in ru_symbols:
    print(s.name)
print()
 

len(*RU*):  2
EURUSD
NEARUSD



In [46]:
# get symbols whose names do not contain USD, EUR, JPY and GBP
group_symbols=mt5.symbols_get(group="*,!*USD*,!*EUR*,!*JPY*,!*GBP*")
print('len(*,!*USD*,!*EUR*,!*JPY*,!*GBP*):', len(group_symbols))
# for s in group_symbols:
#     print(s.name,":",s)

len(*,!*USD*,!*EUR*,!*JPY*,!*GBP*): 1396


In [25]:
symbol_info=mt5.symbol_info("EURJPY")
if symbol_info!=None:
    # display the terminal data 'as is'    
    df=pd.DataFrame(list(symbol_info._asdict().items()),columns=['property','value'])
    print(df)

         property                         value
0          custom                         False
1      chart_mode                             0
2          select                         False
3         visible                         False
4   session_deals                             0
..            ...                           ...
91        formula                              
92           isin                              
93           name                        EURJPY
94           page                              
95           path  Forex\Standard\Majors\EURJPY

[96 rows x 2 columns]


In [34]:
selected=mt5.symbol_select("GBPUSD",True)
print("GBPUSD selected=",selected)

GBPUSD selected= True


In [33]:
lasttick=mt5.symbol_info_tick("GBPUSD")
df=pd.DataFrame(lasttick._asdict(),index=[0])
df['time']=pd.to_datetime(df['time'],unit='s')
print(df)


                 time      bid      ask  last  volume       time_msc  flags  \
0 2024-11-26 10:21:19  1.25494  1.25515   0.0       0  1732616479534   1124   

   volume_real  
0          0.0  


In [45]:
import pytz
from datetime import datetime

pd.set_option('display.max_columns', 20) # number of columns to be displayed
pd.set_option('display.width', 1500)      # max table width to display

timezone = pytz.timezone("Etc/UTC")
utc_from = datetime(2020, 1, 10, tzinfo=timezone)
rates = mt5.copy_rates_from("EURUSD", mt5.TIMEFRAME_H4, utc_from, 10)

for rate in rates:
    print(rate)

(1578484800, 1.11377, 1.11378, 1.11104, 1.11192, 6699, 16, 0)
(1578499200, 1.11192, 1.11301, 1.11081, 1.11172, 7627, 16, 0)
(1578513600, 1.1117, 1.1117, 1.11009, 1.11042, 3412, 16, 0)
(1578528000, 1.11043, 1.11186, 1.11022, 1.11166, 2511, 16, 0)
(1578542400, 1.11166, 1.11182, 1.11119, 1.11175, 1770, 16, 0)
(1578556800, 1.11174, 1.11196, 1.10976, 1.10986, 5889, 16, 0)
(1578571200, 1.10986, 1.11166, 1.10958, 1.11142, 5392, 16, 0)
(1578585600, 1.1114, 1.1114, 1.10918, 1.11039, 5525, 16, 0)
(1578600000, 1.11039, 1.1109, 1.11026, 1.11037, 2657, 16, 0)
(1578614400, 1.11035, 1.11085, 1.10993, 1.11034, 1787, 16, 0)


In [44]:
rates_frame = pd.DataFrame(rates)
rates_frame['time']=pd.to_datetime(rates_frame['time'], unit='s')
print(rates_frame)

                 time     open     high      low    close  tick_volume  spread  real_volume
0 2020-01-08 12:00:00  1.11377  1.11378  1.11104  1.11192         6699      16            0
1 2020-01-08 16:00:00  1.11192  1.11301  1.11081  1.11172         7627      16            0
2 2020-01-08 20:00:00  1.11170  1.11170  1.11009  1.11042         3412      16            0
3 2020-01-09 00:00:00  1.11043  1.11186  1.11022  1.11166         2511      16            0
4 2020-01-09 04:00:00  1.11166  1.11182  1.11119  1.11175         1770      16            0
5 2020-01-09 08:00:00  1.11174  1.11196  1.10976  1.10986         5889      16            0
6 2020-01-09 12:00:00  1.10986  1.11166  1.10958  1.11142         5392      16            0
7 2020-01-09 16:00:00  1.11140  1.11140  1.10918  1.11039         5525      16            0
8 2020-01-09 20:00:00  1.11039  1.11090  1.11026  1.11037         2657      16            0
9 2020-01-10 00:00:00  1.11035  1.11085  1.10993  1.11034         1787      16  

ต่อไปนี้คือลักษณะความแตกต่างระหว่างฟังก์ชันเหล่านี้:

### 1. **`copy_rates_from`**
   - **วัตถุประสงค์**: ฟังก์ชันนี้ใช้ดึงข้อมูลราคาของสัญลักษณ์ที่ระบุ โดยใช้เวลาที่ระบุ (ในกรณีนี้เริ่มต้นจากเวลา `from_time`) และจำนวนแท่งข้อมูล (bars) ที่ต้องการ
   - **การใช้งาน**: ฟังก์ชันนี้จะดึงข้อมูล **price bars** (หรือ OHLC - Open, High, Low, Close) จากช่วงเวลาที่ระบุ
   - **ตัวอย่างการใช้งาน**:
     ```python
     rates = mt5.copy_rates_from(symbol, mt5.TIMEFRAME_M1, from_time, 100)
     ```
   - **พารามิเตอร์**:
     - `symbol`: สัญลักษณ์ (เช่น EURUSD)
     - `timeframe`: ช่วงเวลา (เช่น M1, H1)
     - `from_time`: เวลาที่เริ่มต้น
     - `count`: จำนวนข้อมูลที่ต้องการดึง

### 2. **`copy_rates_from_pos`**
   - **วัตถุประสงค์**: ฟังก์ชันนี้คล้ายกับ `copy_rates_from` แต่จะแตกต่างที่ **จุดเริ่มต้นเป็นตำแหน่ง (position)** แทนการใช้เวลา
   - **การใช้งาน**: ใช้เมื่อคุณต้องการดึงข้อมูลจากตำแหน่งเริ่มต้นใน **history** แทนที่จะใช้เวลา
   - **ตัวอย่างการใช้งาน**:
     ```python
     rates = mt5.copy_rates_from_pos(symbol, mt5.TIMEFRAME_M1, position, 100)
     ```
   - **พารามิเตอร์**:
     - `symbol`: สัญลักษณ์
     - `timeframe`: ช่วงเวลา
     - `position`: ตำแหน่งเริ่มต้น (เช่น 0 คือข้อมูลล่าสุด)
     - `count`: จำนวนแท่งข้อมูล

### 3. **`copy_rates_range`**
   - **วัตถุประสงค์**: ฟังก์ชันนี้ใช้ดึงข้อมูล **ราคา** ในช่วงเวลาที่กำหนด
   - **การใช้งาน**: ใช้เมื่อคุณต้องการดึงข้อมูล **price bars** ตั้งแต่ช่วงเวลาหนึ่งไปจนถึงช่วงเวลาหนึ่ง
   - **ตัวอย่างการใช้งาน**:
     ```python
     rates = mt5.copy_rates_range(symbol, mt5.TIMEFRAME_M1, start_time, end_time)
     ```
   - **พารามิเตอร์**:
     - `symbol`: สัญลักษณ์
     - `timeframe`: ช่วงเวลา
     - `start_time`: เวลาที่เริ่มต้น
     - `end_time`: เวลาที่สิ้นสุด

### 4. **`copy_ticks_from`**
   - **วัตถุประสงค์**: ฟังก์ชันนี้ใช้ดึงข้อมูล **ticks** (ราคาที่เปลี่ยนแปลง) เริ่มจากเวลาที่กำหนด
   - **การใช้งาน**: ใช้ในการดึงข้อมูลราคาที่เปลี่ยนแปลงในลักษณะ **tick** (ไม่ใช่ข้อมูลแบบ bar) ที่มีข้อมูลการเปลี่ยนแปลงราคาในช่วงเวลาหนึ่ง
   - **ตัวอย่างการใช้งาน**:
     ```python
     ticks = mt5.copy_ticks_from(symbol, from_time, 100)
     ```
   - **พารามิเตอร์**:
     - `symbol`: สัญลักษณ์
     - `from_time`: เวลาที่เริ่มต้น
     - `count`: จำนวนข้อมูลที่ต้องการดึง

### 5. **`copy_ticks_range`**
   - **วัตถุประสงค์**: ฟังก์ชันนี้ใช้ดึงข้อมูล **ticks** ในช่วงเวลาที่กำหนด
   - **การใช้งาน**: ใช้เมื่อคุณต้องการดึงข้อมูล **tick** ในช่วงเวลาที่กำหนดระหว่าง `start_time` ถึง `end_time`
   - **ตัวอย่างการใช้งาน**:
     ```python
     ticks = mt5.copy_ticks_range(symbol, start_time, end_time, mt5.COPY_TICKS_ALL)
     ```
   - **พารามิเตอร์**:
     - `symbol`: สัญลักษณ์
     - `start_time`: เวลาที่เริ่มต้น
     - `end_time`: เวลาที่สิ้นสุด
     - `ticks`: สามารถเลือกประเภทของ ticks ที่ต้องการดึง เช่น **`COPY_TICKS_ALL`** หรือ **`COPY_TICKS_INFO`**

---

### สรุปความแตกต่าง:

- **`copy_rates_from`**: ดึงข้อมูลจากเวลาเริ่มต้นและจำนวนแท่งข้อมูลที่ต้องการ
- **`copy_rates_from_pos`**: ดึงข้อมูลจากตำแหน่งเริ่มต้นของข้อมูลที่มี
- **`copy_rates_range`**: ดึงข้อมูลจากช่วงเวลาที่กำหนด
- **`copy_ticks_from`**: ดึงข้อมูล **tick** จากเวลาเริ่มต้น
- **`copy_ticks_range`**: ดึงข้อมูล **tick** จากช่วงเวลาที่กำหนด

การเลือกใช้ฟังก์ชันที่เหมาะสมขึ้นอยู่กับประเภทข้อมูลที่คุณต้องการ (ราคาแบบ bar หรือ tick) และว่าคุณต้องการดึงข้อมูลจากเวลา, ตำแหน่ง, หรือช่วงเวลาใด

หวังว่าคำอธิบายนี้จะช่วยให้คุณเข้าใจความแตกต่างของฟังก์ชันเหล่านี้ครับ! หากยังมีคำถามเพิ่มเติม สามารถถามได้ครับ!